The goal of this file is to vectorize sentences.

## Import

In [7]:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia

In [8]:
# # Install the transformers library
# !pip install transformers

In [ ]:
# !pip install bert-for-tf2

In [ ]:
# !pip install sentencepiece

In [ ]:
# import tensorflow as tf
# import tensorflow_hub as hub 
# from tensorflow.keras import layers 
# import bert 

In [9]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from transformers import BertModel, BertTokenizer

In [10]:
# Load the data
train = pd.read_excel('datasets/all_train_data.xlsx')

## Stop words

In [11]:
# Remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
train['avis_en_cleaned'] = train['avis_en'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))

## Vectorize the data

#### With word2Vec

In [24]:
# Tokenize the data
train['tokenized'] = train['avis_en_cleaned'].apply(lambda x: str(x).split())

# Create the model
model_word2vec = word2vec.Word2Vec(train['tokenized'], min_count=1, vector_size=400)

### With BERT

In [25]:
# Charger le modèle BERT en utilisant la fonction "from_pretrained" de la classe "BertModel"
model_BERT = BertModel.from_pretrained('bert-base-uncased')

# Tokenizer vos données en utilisant la classe "BertTokenizer"
tokenizer_BERT = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##### Même format que ta tokenization précédente

In [ ]:
# tokenized_text = [tokenizer_BERT.tokenize(sent) for sent in train['avis_en_cleaned']]

#### Tokenization BERT avec encode_plus()

In [30]:
# joindre tous les commentaires de manière à avoir un texte
avis_en_cleaned_str = [''.join(i) for i in train['avis_en_cleaned']]
avis_en_cleaned_str = ' '.join(avis_en_cleaned_str)

In [35]:
# Convertir les données tokenizées en tensors en utilisant la fonction "encode_plus" de la classe "BertTokenizer"
encoding_BERT = tokenizer_BERT.encode_plus(avis_en_cleaned_str, 
                                            add_special_tokens = True,    
                                            truncation = True, 
                                            padding = "max_length", 
                                            return_attention_mask = True, 
                                            return_tensors = "pt")

# Générer les embeddings
output = model_BERT(**encoding_BERT)

In [43]:
# Récupérer les embeddings du premier token de la séquence
print(output[0][:, 0, :])

tensor([[-2.5753e-01, -2.1523e-01,  8.6006e-01, -2.6880e-01, -3.4727e-02,
         -3.9694e-03,  7.8232e-01,  1.7451e-01,  1.8797e-01, -5.5796e-01,
          8.5268e-03,  5.1396e-02, -5.3375e-02,  9.6717e-02, -5.8245e-01,
          4.3922e-02,  6.2102e-01,  3.9938e-02, -1.5353e-01,  9.1402e-02,
         -7.8941e-03, -4.2214e-01,  7.8925e-01, -2.3728e-01,  3.1339e-01,
         -4.7162e-01, -4.8409e-01, -3.6018e-01, -4.6085e-01,  4.2710e-01,
          1.8886e-01,  3.4104e-01, -9.0609e-02, -2.3729e-01,  1.7343e-01,
         -1.1408e-01,  5.5881e-01, -3.6389e-02,  2.9742e-01, -1.5055e-01,
          4.8682e-01,  2.8704e-01, -5.9464e-02, -1.4784e-02,  1.1381e-01,
          4.4457e-02, -3.8195e+00,  2.4079e-01, -1.5132e-01, -3.7325e-01,
          5.7998e-01, -7.2593e-01, -3.9227e-01,  1.7995e-01, -7.0363e-03,
          1.0834e-01,  1.1508e-01, -4.8181e-01,  5.3446e-01, -2.7211e-01,
          2.9117e-01, -1.7528e-01,  5.2627e-01,  1.5731e-01,  2.8126e-01,
          2.3845e-01, -1.0989e-01,  8.

##### OPTION CHATGPS qui marche pas (sauf si tu arrive a debug)

In [ ]:
# # Convertir les données tokenizées en tensors en utilisant la fonction "encode_plus" de la classe "BertTokenizer"
# input_ids = tokenizer_BERT.encode_plus(
#     avis_en_cleaned_str,  # Texte à tokenizer
#     max_length=512,  # Longueur maximale de la séquence
#     add_special_tokens=True,  # Ajouter les tokens spéciaux [CLS] et [SEP]
#     return_token_type_ids=True,  # Générer des "token type IDs" pour chaque token
#     pad_to_max_length=True,  # Remplir avec des zéros pour atteindre la longueur maximale
#     return_attention_mask=True  # Générer un masque d'attention pour chaque token
# )

# # Générer les embeddings
# output = model_BERT(**input_ids)

In [ ]:
# Récupérer les embeddings du premier token de la séquence
print(output[0][:, 0, :])

### Compute

In [ ]:
def compute_embedding(text, model):
    return np.mean([model.wv[word] for word in str(text).split()], axis=0)

# Compute the embedding
train['embedding'] = train['avis_en_cleaned'].apply(lambda x: compute_embedding(x, model))

In [ ]:
train

,note,auteur,avis,assureur,produit,type,date_publication,date_exp,avis_en,avis_cor,avis_cor_en,avis_en_cleaned,tokenized,embedding
0,5,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,train,06/09/2021,01/09/2021,"Best insurance, price, solutions, listening, s...",NaN,NaN,"Best insurance, price, solutions, listening, s...","[Best, insurance,, price,, solutions,, listeni...","[0.036845513, -0.09652644, 0.3273503, 0.390465..."
1,4,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,train,03/05/2021,01/05/2021,"I am generally satisfied, except that you have...",NaN,NaN,"I generally satisfied, except problem website,...","[I, generally, satisfied,, except, problem, we...","[0.12592742, -0.17508753, 0.12655705, 0.488717..."
2,5,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,train,21/03/2021,01/03/2021,Very affordable price Several options are avai...,NaN,NaN,Very affordable price Several options availabl...,"[Very, affordable, price, Several, options, av...","[0.09628583, -0.12933695, 0.22045308, 0.241730..."
3,4,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,train,10/06/2021,01/06/2021,"I satisfy the service, a very fast response fr...",NaN,NaN,"I satisfy service, fast response service. I th...","[I, satisfy, service,, fast, response, service...","[0.025221188, -0.10161609, 0.18565398, 0.49751..."
4,1,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,train,29/01/2017,01/01/2017,"Customer for more than 25 years, very disappoi...",NaN,NaN,"Customer 25 years, disappointed ""mutual"" longe...","[Customer, 25, years,, disappointed, ""mutual"",...","[-0.054051913, -0.17967823, 0.14884299, 0.0384..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,4,krys34-96609,J'ai jamais eu de soucis et toujours etait bie...,Assur O'Poil,animaux,train,24/08/2020,01/08/2020,I never had problems and always was well reimb...,NaN,NaN,I never problems always well reimbursed dog ​​...,"[I, never, problems, always, well, reimbursed,...","[-0.032576393, -0.13774587, 0.15680376, 0.2353..."
23996,3,fati-98297,Après 37 ans je quitte la Macif à la suite de ...,MACIF,auto,train,03/10/2020,01/10/2020,After 37 years I leave the Macif following sev...,NaN,NaN,After 37 years I leave Macif following several...,"[After, 37, years, I, leave, Macif, following,...","[0.08294767, -0.14988041, 0.07407212, 0.309009..."
23997,1,jeremy59-96188,Je suis propriétaire d'une flotte de véhicules...,AXA,flotte-automobile,train,11/08/2020,01/08/2020,I own a fleet of motor vehicles which was insu...,NaN,NaN,I fleet motor vehicles insured Axa Insurance m...,"[I, fleet, motor, vehicles, insured, Axa, Insu...","[-0.084414475, -0.19338982, 0.20123054, 0.0992..."
23998,1,mic-54375,Mutuelle à fuir. Chère. Remboursements insuffi...,Mgen,sante,train,29/04/2017,01/04/2017,Mutual to flee. Dear. Insufficient reimburseme...,NaN,NaN,Mutual flee. Dear. Insufficient reimbursements...,"[Mutual, flee., Dear., Insufficient, reimburse...","[-0.06431994, -0.14848596, 0.12282439, -0.0018..."


In [ ]:
# Save the data in pickle format
train.to_pickle('datasets/all_train_data_embedding.pkl')

In [ ]:
# Save the model
model.save('models/word2vec.model')